# BRAZ Lucas & DURAND Pierre-Alain
## SCIPER: 343141 & SCIPER: 344313

In [120]:
import numpy as np
import pandas as pd

import random

import multiprocessing as mp
from joblib import Parallel, delayed

from tic_env import TictactoeEnv, OptimalPlayer

import plotly.express as px
import plotly.graph_objects as go

# 2 *Q*-Learning

In [84]:
def play_games(player_opt, agent, maxGames=20_000, env=TictactoeEnv()):
    Turns = np.array(['X','O'])
    winnerList = np.zeros(maxGames)
    for nbGames in range(maxGames):
        env.reset()
        grid, _, __ = env.observe()

        player_opt.player = Turns[nbGames%2]
        agent.player = Turns[(nbGames+1)%2]

        for roundGame in range(9):
            if env.current_player == player_opt.player:
                if roundGame > 1 and isinstance(player_opt, QlearningAgent):
                    player_opt.learn(grid, 0)
                move = player_opt.act(grid)
            else:
                if roundGame > 1 and isinstance(agent, QlearningAgent):
                    agent.learn(grid, 0)
                move = agent.act(grid)            

            grid, end, winner = env.step(move, print_grid=False)

            if end:
                if winner == agent.player:
                    winnerList[nbGames] = 1
                    if isinstance(player_opt, QlearningAgent):
                        player_opt.learn(grid, -1, end=True)
                    if isinstance(agent, QlearningAgent):
                        agent.learn(grid, 1, end=True)
                elif winner == player_opt.player:
                    winnerList[nbGames] = -1
                    if isinstance(player_opt, QlearningAgent):
                        player_opt.learn(grid, 1, end=True)
                    if isinstance(agent, QlearningAgent):
                        agent.learn(grid, -1, end=True)
                else:
                    if isinstance(player_opt, QlearningAgent):
                        player_opt.learn(grid, 0, end=True)
                    if isinstance(agent, QlearningAgent):
                        agent.learn(grid, 0, end=True)
                break
    env.reset()
    return winnerList

In [85]:
def play_games_with_M(player_opt, agent, maxGamesTotal=20_000, deltaM=250, env=TictactoeEnv()):
    Mopt = []
    Mrandom = []
    player_realOpt = OptimalPlayer(epsilon=0.0)
    player_random = OptimalPlayer(epsilon=1.0)
    winnerList = []

    for _ in range(maxGamesTotal//deltaM):
        winnerList.append(play_games(player_opt, agent, deltaM))

        currentEpsilon = agent.epsilon
        agent.epsilon = 0
        agent.isLearning = False

        Mopt.append(play_games(player_realOpt, agent, 500).mean())
        Mrandom.append(play_games(player_random, agent, 500).mean())

        agent.isLearning = True
        agent.epsilon = currentEpsilon
    winnerList = np.concatenate(winnerList)
    return winnerList, Mopt, Mrandom

In [6]:
class Qtable:
    def __init__(self, Qtab={}, defaultValue=0.0):
        self.Qtab = Qtab
        self.defaultValue = defaultValue

    def hash(self, state_action):
        """
        hash a state-action tuple to a unique integer. Each element of the grid (state) has 3 states, 
        same for the action position.
        Then, we consider the state-action pair as a base 3 number, and we hash it to an integer.
        Can be easily recover from hash to state-action pair.
        """
        state, action = state_action

        hashState = (state.flatten()+1)@np.array([3**0,3**1,3**2,3**3,3**4,3**5,3**6,3**7,3**8])
        hashAction = action[0]*3**9 + action[1]*3**10
        return hashState + hashAction
    
    def __getitem__(self, key: tuple):
        return self.Qtab.get(self.hash(key), self.defaultValue)

    def __setitem__(self, key: tuple, value):
        self.Qtab[self.hash(key)] = value

In [7]:
class QlearningAgent:
    """
    Description:
        A class to implement an epsilon-greedy Qlearning Agent in Tic-tac-toe.

    Parameters:
        epsilon: float, in [0, 1]. This is a value between 0-1 that indicates the
            probability of making a random action instead of the greedy action
            at any given time.

        learningRate: float, in [0, 1]. Setting it to 0 means that the Q-values are
            never updated, hence nothing is learned. Setting a high value such as 0.9 
            means that learning can occur quickly. 
        
        discountFactor: float, in [0, 1]. This models the fact that future rewards are
            worth less than immediate rewards. Setting it to 0 means that the agent
            will only learn from immediate rewards. Setting it to 1 means that the
            agent will learn from all rewards equally.
    """

    def __init__(self, epsilon=0.2, player='X', learningRate=0.05, discountFactor=0.99, Q={}, Q_defaultValue=0.0, n_max=100):
        if isinstance(epsilon, tuple):
            self.epsilon_min, self.epsilon_max = epsilon
            self.epsilon = self.epsilon_max
        else:
            self.epsilon = epsilon
            self.epsilon_min = epsilon
            self.epsilon_max = epsilon
        self.learningRate = learningRate
        self.discountFactor = discountFactor

        self.Q = Qtable(Q, Q_defaultValue)
        self.state = None
        self.action = None

        self.n = 0
        self.n_max = n_max

        self.isLearning = True

        self.player = player # 'X' or 'O'

    def decrease_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon_max * (1 - self.n / self.n_max))


    def set_player(self, player = 'X', j=-1):
        self.player = player
        if j != -1:
            self.player = 'X' if j % 2 == 0 else 'O'

    def empty(self, state):
        """ Return all empty positions. """
        availableActions = []
        for x in range(3):
            for y in range(3):
                position = (x, y)
                if state[position] == 0:
                    availableActions.append(position)
        return availableActions

    def randomAction(self, state):
        """ Choose a random action from the available options. """
        availableActions = self.empty(state)

        return random.choice(availableActions)

    def bestAction(self, state):
        """
        Choose the available actions which have a maximum expected future reward. 
        If there are multiple actions with the same maximum expected future reward,
        choose one of them at random.
        """
        # Get the available moves
        availableActions = self.empty(state)

        # Get the best move
        bestActions = []
        bestValue = -999.0
        for action in availableActions:
            Qsa = self.Q[(state, action)]
            if Qsa == bestValue:
                bestActions.append(action)
            if Qsa > bestValue:
                bestActions = [action]
                bestValue = Qsa

        return random.choice(bestActions)

    def act(self, state):
        """
        epsilon-greedy action selection, according to the Q-table.
        """
        self.state = state

        # whether move in random or not
        if random.random() < self.epsilon:
            self.action = self.randomAction(state)
        else:
            # Get the best move
            self.action = self.bestAction(state)

        return self.action

    def learn(self, s_prime, reward, end=False):
        """
        Q-learning update. If it's the end of a game, we set Q(s',a') = 0.
        """
        if self.isLearning:
            if not end:
                # Get the best move
                a_prime = self.bestAction(s_prime)

                # Update the Q-value
                self.Q[(self.state, self.action)] += self.learningRate * (reward + self.discountFactor * self.Q[(s_prime, a_prime)] - self.Q[(self.state, self.action)])
            else:
                self.Q[(self.state, self.action)] += self.learningRate * (reward - self.Q[(self.state, self.action)])

                self.state = None
                self.action = None

                self.n += 1
                self.decrease_epsilon()

## 2.1 Learning from experts

#### Question 1

In [86]:
epsilon = 0.4

player_opt = OptimalPlayer(epsilon=0.5)
agent = QlearningAgent(epsilon=epsilon)

winnerList = play_games(player_opt, agent, maxGames=20_000)

groupSize = 250
y=winnerList.reshape(winnerList.size//groupSize, groupSize).mean(axis=1)
x=np.arange(y.size)*groupSize

fig = px.line(x=x, y=y, title=f'Average reward over time of RL agent with policy epsilon={epsilon}')
fig.update_layout(xaxis_title='Game number', yaxis_title='Average reward')
fig.show()

### 2.1.1 Decreasing exploration

#### Question 2

In [33]:
n_max_list = [1, 100, 1_000, 10_000, 20_000, 30_000, 40_000]

epsilon = (0.1, 0.8)

maxGamesTotal = 20_000
groupSize = 250
df = {'Game number':np.arange(maxGamesTotal//groupSize)*groupSize}

player_opt = OptimalPlayer(epsilon=0.5)

num_cores = min(len(n_max_list),mp.cpu_count())

def parallelGames(n_max, player_opt, epsilon, maxGamesTotal, groupSize):
    df = {}

    agent = QlearningAgent(epsilon=epsilon, n_max=n_max)

    winnerList = play_games(player_opt, agent, maxGames=maxGamesTotal)

    y=winnerList.reshape(winnerList.size//groupSize, groupSize).mean(axis=1)

    df[f'Average reward (n*={n_max})'] = y

    return df

dfs = Parallel(n_jobs=num_cores)(delayed(parallelGames)(n_max, player_opt, epsilon, maxGamesTotal, groupSize) for n_max in n_max_list)

for d in dfs:
    df.update(d)

df = pd.DataFrame(df)
fig = px.line(df, x='Game number', y=df.columns.difference(['Game number']), title=f'Average reward over time of RL agent with policy epsilon={epsilon}')
fig.show()

#### Question 3

In [87]:
n_max_list = [1, 1_000, 10_000, 40_000]

epsilon = (0.1, 0.8)

maxGamesTotal = 20_000
groupSize = 250
df = {'Game number':np.arange(maxGamesTotal//groupSize)*groupSize}


num_cores = min(len(n_max_list),mp.cpu_count())

def parallelGames(n_max, epsilon, maxGamesTotal, groupSize):
    df = {}

    player_opt = OptimalPlayer(epsilon=0.5)
    agent = QlearningAgent(epsilon=epsilon, n_max=n_max)

    winnerList, Mopt, Mrandom = play_games_with_M(player_opt, agent, maxGamesTotal, groupSize)

    # y = winnerList.reshape(winnerList.size//groupSize, groupSize).mean(axis=1)

    #df[f'Average reward (n*={n_max})'] = y
    df[f'Mopt (n*={n_max})'] = Mopt
    df[f'Mrandom (n*={n_max})'] = Mrandom

    return df

dfs = Parallel(n_jobs=num_cores)(delayed(parallelGames)(n_max, epsilon, maxGamesTotal, groupSize) for n_max in n_max_list)

for d in dfs:
    df.update(d)

df = pd.DataFrame(df)
fig = px.line(df, x='Game number', y=df.columns.difference(['Game number']), title=f'Average reward over time of RL agent with policy epsilon={epsilon}')
fig.show()

### 2.1.2 Good experts and bad experts

#### Question 4

In [69]:
epsilonOptList = [0, 0.25, 0.5, 0.75, 1.0]

n_max = 1 

epsilon = (0.1, 0.8)

maxGamesTotal = 20_000
groupSize = 250
df = {'Game number':np.arange(maxGamesTotal//groupSize)*groupSize}
df2 = {'Game number':np.arange(maxGamesTotal//groupSize)*groupSize}

num_cores = mp.cpu_count()

def parallelGames(epsilonOpt, maxGamesTotal, groupSize, epsilon, n_max):
    df = {}
    df2 = {}

    agent = QlearningAgent(epsilon=epsilon, n_max=n_max)
    player_opt = OptimalPlayer(epsilon=epsilonOpt)

    winnerList, Mopt, Mrandom = play_games_with_M(player_opt, agent, maxGamesTotal, groupSize)

    y = winnerList.reshape(winnerList.size//groupSize, groupSize).mean(axis=1)

    df[f'Average reward (epsilonOpt={epsilonOpt})'] = y
    df2[f'Mopt (epsilonOpt={epsilonOpt})'] = Mopt
    df2[f'Mrandom (epsilonOpt={epsilonOpt})'] = Mrandom

    return df,df2

dfs = Parallel(n_jobs=num_cores)(delayed(parallelGames)(epsilonOpt, maxGamesTotal, groupSize, epsilon, n_max) for epsilonOpt in epsilonOptList)

for d,d2 in dfs:
    df.update(d)
    df2.update(d2)

df = pd.DataFrame(df)
fig = px.line(df, x='Game number', y=df.columns.difference(['Game number']), title=f'Average reward over time of RL agent with policy epsilon={epsilon}')
fig.show()

#### Question 5

In [81]:
df2 = pd.DataFrame(df2)
fig = px.line(df2, x='Game number', y=df2.columns.difference(['Game number']), title=f'Average reward over time of RL agent with policy epsilon={epsilon}')
fig.show()

#### Question 6

TODO

## 2.2 Learning by self-practice

#### Question 7

In [ ]:
epsilonList = [0, 0.25, 0.5, 0.75, 1.0]

n_max = 1 

maxGamesTotal = 20_000
groupSize = 250
df = {'Game number':np.arange(maxGamesTotal//groupSize)*groupSize}

num_cores = mp.cpu_count()

def parallelGames(epsilon, maxGamesTotal, groupSize, n_max):
    df = {}

    agent = QlearningAgent(epsilon=epsilon, n_max=n_max)
    player_opt = QlearningAgent(epsilon=agent.epsilon, learningRate=agent.learningRate, discountFactor=agent.discountFactor, Q={}, Q_defaultValue=agent.Q_defaultValue, n_max=agent.n_max)
    player_opt.Q = agent.Q

    _, Mopt, Mrandom = play_games_with_M(player_opt, agent, maxGamesTotal, groupSize)

    df[f'Mopt (epsilonOpt={epsilon})'] = Mopt
    df[f'Mrandom (epsilonOpt={epsilon})'] = Mrandom

    return df

dfs = Parallel(n_jobs=num_cores)(delayed(parallelGames)(epsilon, maxGamesTotal, groupSize, n_max) for epsilon in epsilonList)

for d in dfs:
    df.update(d)

df = pd.DataFrame(df)
fig = px.line(df, x='Game number', y=df.columns.difference(['Game number']), title=f'Average reward over time of RL agent with policy epsilon={epsilon}')
fig.show()

#### Question 8

In [ ]:
epsilonList = [(0.1, 0.8), (0.0, 1.0), (0.2, 0.7), (0.4, 1.0)]

n_max = 1 

maxGamesTotal = 20_000
groupSize = 250
df = {'Game number':np.arange(maxGamesTotal//groupSize)*groupSize}

num_cores = mp.cpu_count()

def parallelGames(epsilon, maxGamesTotal, groupSize, n_max):
    df = {}

    agent = QlearningAgent(epsilon=epsilon, n_max=n_max)
    player_opt = QlearningAgent(epsilon=agent.epsilon, learningRate=agent.learningRate, discountFactor=agent.discountFactor, Q={}, Q_defaultValue=agent.Q_defaultValue, n_max=agent.n_max)
    player_opt.Q = agent.Q

    _, Mopt, Mrandom = play_games_with_M(player_opt, agent, maxGamesTotal, groupSize)

    df[f'Mopt (epsilonOpt={epsilon})'] = Mopt
    df[f'Mrandom (epsilonOpt={epsilon})'] = Mrandom

    return df

dfs = Parallel(n_jobs=num_cores)(delayed(parallelGames)(epsilon, maxGamesTotal, groupSize, n_max) for epsilon in epsilonList)

for d in dfs:
    df.update(d)

df = pd.DataFrame(df)
fig = px.line(df, x='Game number', y=df.columns.difference(['Game number']), title=f'Average reward over time of RL agent with policy epsilon={epsilon}')
fig.show()

#### Question 9

TODO

#### Question 10

In [130]:
state = np.array([[0, 1, 0], [0, 0, 0], [0, 0, 0]])

fig = px.imshow([[agent.Q[(state,(0,0))], agent.Q[(state,(0,1))], agent.Q[(state,(0,2))]],
                 [agent.Q[(state,(1,0))], agent.Q[(state,(1,1))], agent.Q[(state,(1,2))]],
                 [agent.Q[(state,(2,0))], agent.Q[(state,(2,1))], agent.Q[(state,(2,2))]]], color_continuous_scale='gray',range_color=(0,1),text_auto=True)
fig.show()

In [105]:
state = np.array([[1, 0, 0], [0, 1, 0], [0, -1, 0]])

fig = px.imshow([[agent.Q[(state,(0,0))], agent.Q[(state,(0,1))], agent.Q[(state,(0,2))]],
                 [agent.Q[(state,(1,0))], agent.Q[(state,(1,1))], agent.Q[(state,(1,2))]],
                 [agent.Q[(state,(2,0))], agent.Q[(state,(2,1))], agent.Q[(state,(2,2))]]], color_continuous_scale='gray',range_color=(0,1))
fig.show()

In [102]:
state = np.array([[0, -1, 0], [0, 1, 0], [0, 1, 0]])

fig = px.imshow([[agent.Q[(state,(0,0))], agent.Q[(state,(0,1))], agent.Q[(state,(0,2))]],
                 [agent.Q[(state,(1,0))], agent.Q[(state,(1,1))], agent.Q[(state,(1,2))]],
                 [agent.Q[(state,(2,0))], agent.Q[(state,(2,1))], agent.Q[(state,(2,2))]]], color_continuous_scale='gray',range_color=(0,1))
fig.show()

.